In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from xgboost import XGBRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

import plotly.express as px
from plotly.offline import *
from plotly.subplots import make_subplots

import random

In [3]:
repo = ('https://raw.githubusercontent.com/PolarSprings/Iowa_advanced_regression/main/model_selec_dim.csv?token=AKXGRRTFSDYTOOXF5CLCQJDA4OZCE')

models3 = pd.read_csv(repo, skipinitialspace=True)

In [4]:
data = models3.drop(['AdjSalePrice'], 1)
target = models3[['AdjSalePrice']].squeeze()

In [99]:
# xgbooster = xgb.XGBRegressor(verbosity=1)

# data = models3.drop(['AdjSalePrice'], 1)
# target = models3[['AdjSalePrice']].squeeze()

# idx = np.random.choice(range(0,len(models3)), size=round(len(models3)*0.75), replace=False)

# x_train = data.iloc[idx]
# y_train = target.iloc[idx]

# x_test = data.iloc[~idx]
# y_test = target.iloc[~idx]

# xgbooster.fit(x_train, y_train)

# print('the train score for this model is :', xgbooster.score(x_train, y_train))
# print('the test score for this model is :', xgbooster.score(x_test, y_test))

the train score for this model is : 0.9969275290843757
the test score for this model is : 0.9530533594875293


In [5]:
boost = xgb.XGBRegressor(verbosity=0)

boost.fit(data, target)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [8]:
params = {
    'objective':['reg:squarederror'],
    'n_estimators':[300, 400, 500],
#     'min_child_weight': [5, 6, 7],
#     'max_depth':[5],
#     'gamma': [0,0.5, 1],
#     'lambda':[0, 0.0001, 0.5, 1],
#     'alpha':[0, 0.5, 1],
    'feature_selector':['greedy'],
#     'eta':[0, 0.5, 1],
#     'colsample_bytree': [0.3, 0.4, 0.5],
#     'max_delta_step':[0],
#     'refresh_leaf':[0],
#     'tree_method':['auto'],
#     'sketch_eps':[0],
    'updater':['grow_colmaker'],
#     'process_type':['default']
}

In [9]:
random_search = RandomizedSearchCV(boost, param_distributions=params, n_jobs=-1, cv=2, n_iter=128).fit(data, target)

c:\python38\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning:

The total space of parameters 3 is smaller than n_iter=128. Running 3 iterations. For exhaustive searches, use GridSearchCV.



In [10]:
random_search.best_params_

{'updater': 'grow_colmaker',
 'objective': 'reg:squarederror',
 'n_estimators': 300,
 'feature_selector': 'greedy'}

In [12]:
grid_search = GridSearchCV(boost, param_grid=params, n_jobs=-1, cv=2).fit(data, target)

In [13]:
grid_search.best_params_

{'feature_selector': 'greedy',
 'n_estimators': 300,
 'objective': 'reg:squarederror',
 'updater': 'grow_colmaker'}

In [15]:
params = {
    'objective':'reg:squarederror',
    'n_estimators':300,
      'max_delta_step':0,
    'updater':'grow_colmaker',
}


In [16]:
xgbooster = xgb.XGBRegressor(verbosity=1)

xgbooster.set_params(**params)

xgbooster.fit(data, target)

[21:32:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/gbm/gbtree.cc:82: DANGER AHEAD: You have manually specified `updater` parameter. The `tree_method` parameter will be ignored. Incorrect sequence of updaters will produce undefined behavior. For common uses, we recommend using `tree_method` parameter instead.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=300, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='auto', updater='grow_colmaker', validate_parameters=1,
             verbosity=1)

In [17]:
idx = np.random.choice(range(0,len(models3)), size=round(len(models3)*0.75), replace=False)

x_train = data.iloc[idx]
y_train = target.iloc[idx]

x_test = data.iloc[~idx]
y_test = target.iloc[~idx]

xgbooster.fit(x_train, y_train)

print('the train score for this model is :', xgbooster.score(x_train, y_train))
print('the test score for this model is :', xgbooster.score(x_test, y_test))

[21:32:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/gbm/gbtree.cc:82: DANGER AHEAD: You have manually specified `updater` parameter. The `tree_method` parameter will be ignored. Incorrect sequence of updaters will produce undefined behavior. For common uses, we recommend using `tree_method` parameter instead.
the train score for this model is : 0.9999063728337865
the test score for this model is : 0.971153112180821
